In [ ]:
#default_exp data.make_sample_data

In [ ]:
!nbdev_build_lib

Converted 00_jobs.ipynb.
Converted 01-create-sample-data.ipynb.
Converted 02-preprocess.ipynb.
Converted 03-feature-calc.ipynb.
Converted 04-training-data.ipynb.
Converted 05-train-model.ipynb.
Converted 06-submit-training-pipeline.ipynb.
Converted 99-tools.ipynb.
Converted index.ipynb.


In [ ]:
#export
from datetime import datetime, timezone, timedelta
import random
import uuid
import hashlib
import os
from copy import copy

import numpy as np
import pandas as pd

from hopeit.app.context import EventContext
from hopeit.app.events import Spawn, SHUFFLE
from hopeit.app.api import event_api
from hopeit.app.logger import app_logger

from fraud_poc.jobs import get_client_async, MakeSampleDataJob

In [ ]:
#export
__steps__ = ['submit_job', SHUFFLE, 'make_batches']

__api__ = event_api(
    title="Data: Make sample data",
    query_args=[
        ("num_batches", int, "Number of batches/files to make"),
        ("batch_size", int, "Number of rows per batch"),
        ("batch_span_days", int, "Number of time span in days for order_date in batch"),
        ("num_customers", int, "Number of customers to generate"),
        ("num_emails", int, "Number of emails to generate"),
        ("num_ips", int, "Number of IP addresses to generate")
    ],
    responses={
        200: (MakeSampleDataJob, "Job submitted")
    }
)

logger = app_logger()

In [ ]:
#export
class OrderRandomSource:
    @staticmethod
    def new_uuid(): return str(uuid.uuid4())
    
    @staticmethod
    def new_hash(): return hashlib.sha1(str(uuid.uuid4()).encode()).hexdigest()

    def __init__(self, days_ago: int, days_span: int,
                 customer_ids: list, emails: list, email_to_customer: list, ips, email_to_ip: list):
        self.num_customers = len(customer_ids)
        self.num_ips = len(ips)
        self.num_emails = len(emails)
        self.lat_min, self.lat_max = -20., -10.
        self.long_min, self.long_max = -20., -10.
        self.to_date = datetime.now(tz=timezone.utc) - timedelta(days=days_ago)
        self.to_date_epoch = int(self.to_date.timestamp())
        self.from_date = self.to_date - timedelta(days=days_span)
        self.from_date_epoch = int(self.from_date.timestamp())
        self.customer_ids = customer_ids
        self.emails = emails
        self.email_to_customer = email_to_customer
        self.ips = ips
        self.email_to_ip = email_to_ip
        
    def random_order(self):
        email_idx = random.randint(0, self.num_emails-1)
        email = self.emails[email_idx]
        customer_id = self.email_to_customer[email_idx]
        ip_addr = self.email_to_ip[email_idx] if random.random() > 0.1 else self.new_hash()
        date_epoch = random.randint(self.from_date_epoch, self.to_date_epoch)
        date = datetime.fromtimestamp(date_epoch, tz=timezone.utc)
        lat = self.lat_min + random.random() * (self.lat_max - self.lat_min)
        long = self.long_min + random.random() * (self.long_max - self.long_min)
        amount = random.random() * 1000.
        return {
            'order_id': self.new_uuid(),
            'order_date': date,
            'customer_id': customer_id,
            'email': email,
            'ip_addr': ip_addr,
            'location_lat': lat,
            'location_long': long,
            'order_amount': amount
        }

    def _generate_orders(self, n: int):
        for _ in range(n):
            yield self.random_order()

    def __call__(self, n: int):
        return pd.DataFrame(self._generate_orders(n))

    @staticmethod
    def meta(): 
        return {'order_id': object, 
               'order_date': 'datetime64[ns, UTC]', 
               'customer_id': object, 
               'email': object, 
               'ip_addr': object, 
               'location_lat': float, 
               'location_long': float, 
               'order_amount': float}

In [ ]:
#export
def _make_batch(path: str, i: int, size: int, days_span: int,
               customer_ids: list, emails: list, email_to_customer: list, ips, email_to_ip: list):
    os.makedirs(path, exist_ok=True)
    days_ago = i * days_span
    random_orders = OrderRandomSource(days_ago=days_ago, days_span=days_span,
                                      customer_ids=customer_ids, emails=emails, email_to_customer=email_to_customer, 
                                      ips=ips, email_to_ip=email_to_ip)
    df = random_orders(size) #, meta=random_orders.meta())
    file_name = f'{path}/batch{i:02}.parquet'
    df.to_parquet(file_name, engine='fastparquet', compression='LZ4')
    return file_name

In [ ]:
#export
async def submit_job(payload: None, context: EventContext, 
                     num_batches: int = 12, batch_size: int = 100000, batch_span_days: int = 30,
                     num_customers: int = 10000, num_emails: int = 10000, num_ips: int = 10000) -> MakeSampleDataJob:
    path = context.env['data']['raw']
    return MakeSampleDataJob(path, int(num_batches), int(batch_size), int(batch_span_days),
                            int(num_customers), int(num_emails), int(num_ips))

async def make_batches(job: MakeSampleDataJob, context: EventContext) -> MakeSampleDataJob:
    logger.info(context, f"Executing: {job}...")
    client = await get_client_async(context)
    logger.info(context, f"Dask: {client}")
    try:
        batches = []
        customer_ids = [OrderRandomSource.new_uuid() for i in range(job.num_customers)]
        emails = [OrderRandomSource.new_hash() for i in range(job.num_emails)]
        email_to_customer = [customer_ids[random.randint(0, job.num_customers-1)] for i in range(job.num_emails)]
        ips = [OrderRandomSource.new_hash() for i in range(job.num_ips)]
        email_to_ip = [ips[random.randint(0, job.num_ips-1)] for i in range(job.num_emails)] 

        for i in range(job.num_batches):
            logger.info(context, f"Submitting batch {i}...")
            batches.append(
                client.submit(_make_batch, job.path, i, job.batch_size, job.batch_span_days,
                             customer_ids, emails, email_to_customer, ips, email_to_ip)
            )

        for batch in batches:
            res = await batch
            logger.info(context, f"Done batch: {res}.")
        #dfs = await client.gather(*batches)
        return job
    except Exception as e:
        logger.error(context, e)
        return None
    finally:
        await client.close()

### Test from notebook

In [ ]:
from hopeit.testing.apps import config, execute_event

app_config = config('config/training-pipeline.json')
result = await execute_event(app_config, 'data.make-sample-data', None,
                            num_batches=2, batch_size=1000, batch_span_days=10,
                            num_customers=1000, num_emails=1000, num_ips=1000)
result

2020-07-07 07:58:55,374 | INFO | fraud-poc 0.0.1 data.make-sample-data leo-legion 9542 | Executing: MakeSampleDataJob(path='./data/raw', num_batches=2, batch_size=1000, batch_span_days=10, num_customers=1000, num_emails=1000, num_ips=1000)... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-07T07:58:55.373982+00:00
2020-07-07 07:58:56,222 | INFO | fraud-poc 0.0.1 data.make-sample-data leo-legion 9542 | Dask: <Client: 'tcp://127.0.0.1:35077' processes=4 threads=12, memory=33.54 GB> | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-07T07:58:55.373982+00:00
2020-07-07 07:58:56,246 | INFO | fraud-poc 0.0.1 data.make-sample-data leo-legion 9542 | Submitting batch 0... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-07T07:58:55.373982+00:00
2020-07-07 07:58:56,255 | INFO | fraud-poc 0.0.1 data.make-sample-data leo-legion 9542 | Submitting

MakeSampleDataJob(path='./data/raw', num_batches=2, batch_size=1000, batch_span_days=10, num_customers=1000, num_emails=1000, num_ips=1000)

In [ ]:
import dask.dataframe as dd
dd.read_parquet(result.path).head()

,order_id,order_date,customer_id,email,ip_addr,location_lat,location_long,order_amount
0,d03d2ac3-e743-4d36-89b3-69ada8bc644f,2020-07-01 05:56:22+00:00,d0028e6c-877d-485b-9729-db7d25bb30e9,0b083a8a6297fb87c3896a8646829e9abdf03586,481a69203c335d1793b3064588318c2502685b96,-10.338813,-11.187772,560.495050
1,1e0fa14e-e4dd-4679-984a-4ff55a45203a,2020-06-29 03:03:09+00:00,8b5eadd0-9385-402f-a8de-19280e2f2e25,9276818c1d9ade25b4f5b8379edff60a02aa3b20,d4ce28f637554a62f9a2a71a0ebbd4993fe9a78b,-13.593945,-11.779278,952.575210
2,034373df-10ab-4c41-afb2-84a2e8b7c2c6,2020-07-01 01:26:53+00:00,e8c4e465-0f16-4c7e-9d30-0ce0e4018b90,765d0eae8a3017bca98ef69c00ef6cf8a9b8f758,a3c6de1868c82ba38cf50098b4fe884c1ed31f2c,-12.475382,-10.575640,935.767647
3,11bc1132-88f5-40bf-9269-664815636ca4,2020-06-27 11:16:25+00:00,8392e44d-6cb9-4d6a-8597-025a37b3932e,41017cdbcf7672f4e112750756280da5410496c5,216e248199e3d65a12de3a25c355c9856072432c,-13.095234,-11.961017,463.160127
4,c92cb30b-991f-4568-a747-f4f6165f6e3b,2020-07-04 11:01:48+00:00,d2112e14-b65b-460d-b525-657b535aa05a,efa842b9609a522ec496262834937404fc1029d4,c2b9c97ed4a461ab3cf70cd57176bd0308bff910,-17.466511,-13.253921,794.789987
